# Synthesis Inspection

Display synthesizer input alongside final summary and verify Output_Framework section coverage.

In [ ]:
import sys
import json
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
from IPython.display import Markdown, display
from src.config import OUTPUT_DIR, OUTPUT_FRAMEWORK_FILE
from src.schemas import InterpreterOutput, SynthesizerOutput
from src.synthesizer import synthesize

## Load Interpretation Results

In [ ]:
interp_path = OUTPUT_DIR / "interpretation.json"
data = json.loads(interp_path.read_text(encoding="utf-8"))
interpreter_output = InterpreterOutput.model_validate(data)

print(f"Reconciled findings: {len(interpreter_output.reconciled_findings)}")
print(f"Conflicts resolved:  {len(interpreter_output.conflicts_resolved)}")
print(f"Duplicates removed:  {interpreter_output.duplicates_removed}")

## Input: Reconciled Findings

In [ ]:
rows = []
for i, rf in enumerate(interpreter_output.reconciled_findings, 1):
    sources = ", ".join(rf.sources)
    content_preview = rf.content[:120] + ("…" if len(rf.content) > 120 else "")
    rows.append(f"| {i} | {rf.datetime or '—'} | {content_preview} | {sources} |")

table = (
    "| # | Datetime | Content | Sources |\n"
    "|---|----------|---------|---------|\n"
    + "\n".join(rows)
)
display(Markdown(table))

## Run Synthesizer

In [ ]:
synthesizer_output = await synthesize(interpreter_output)

print(f"Findings incorporated: {synthesizer_output.metadata.findings_incorporated}")
print(f"Date range: {synthesizer_output.metadata.date_range}")

## Output: Clinical Summary

In [ ]:
display(Markdown(synthesizer_output.summary))

## Output Framework Coverage

In [ ]:
framework_sheets = pd.read_excel(OUTPUT_FRAMEWORK_FILE, engine="openpyxl", sheet_name=None)
expected_sections = list(framework_sheets.keys())

summary_text = synthesizer_output.summary.lower()
coverage_rows = []
for section in expected_sections:
    found = section.lower() in summary_text
    coverage_rows.append({"Section": section, "Found in Summary": found})

coverage_df = pd.DataFrame(coverage_rows)
found_count = coverage_df["Found in Summary"].sum()
total = len(coverage_df)
print(f"Section coverage: {found_count}/{total} ({found_count/total*100:.0f}%)\n")
display(coverage_df)

## Save Results

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
summary_path = OUTPUT_DIR / "summary.md"
summary_path.write_text(synthesizer_output.summary, encoding="utf-8")
print(f"Saved: {summary_path}")